<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# IBM Watson OpenScale and Batch Processing:<br>Apache Spark on Cloud Pak for Data with IBM Analytics Engine

This notebook must be run in the Python 3.10 runtime environment. It requires Watson OpenScale service credentials.

The notebook configures Watson OpenScale to monitor the German Credit Risk model. Use the notebook to enable quality, drift, drift_v2, fairness and explainability monitoring and run on-demand evaluations. Before you can run the notebook, you must have the following resources:

1. The common configuration JSON, Drift, Drift V2 Configuration and Explainability Configuration archives generated by using the [common configuration notebook](https://github.com/IBM/watson-openscale-samples/blob/main/Cloud%20Pak%20for%20Data/Batch%20Support/Configuration%20generation%20for%20OpenScale%20batch%20subscription.ipynb).
2. Feedback, payload, drifted transactions, explanations queue and result tables in a supported database (DB2 or Postgres) storage that use the data description language statements (DDLs) that are generated as part of running the previous common configuration notebook.

In [1]:
# ----------------------------------------------------------------------------------------------------
# IBM Confidential
# OCO Source Materials
# 5737-H76
# Copyright IBM Corp. 2021-2025
# The source code for this Notebook is not published or other-wise divested of its trade
# secrets, irrespective of what has been deposited with the U.S.Copyright Office.
# ----------------------------------------------------------------------------------------------------

VERSION = "iae-jdbc-1.0"

# Version History
# iae-jdbc-1.0 : Add partitioning information
# <no-verion>  : Initial release

## Contents

1. [Setup](#setup)
2. [Configure Watson OpenScale](#openscale)
3. [Set up a subscription](#subscription)
4. [Quality monitoring](#quality)
5. [Drift monitoring](#drift)
6. [Drift V2 monitoring](#drift_v2)
6. [Fairness monitoring](#fairness)
7. [Explainability monitoring](#explainability)

# 1. Setup <a name="setup"></a>

## Package installation

First import some of the packages you need to use. After you finish installing the following software packages, restart the kernel.

In [2]:
import warnings
warnings.filterwarnings("ignore")
%env PIP_DISABLE_PIP_VERSION_CHECK=1

env: PIP_DISABLE_PIP_VERSION_CHECK=1


In [3]:
!pip install -U ibm-watson-openscale --no-cache | tail -n 1

In [4]:
!pip show ibm-watson-openscale

Name: ibm-watson-openscale
Version: 3.0.41
Summary: Client library for IBM Watson OpenScale
Home-page: https://github.ibm.com/watson-developer-cloud/openscale-python-sdk
Author: IBM Watson OpenScale
Author-email: kishore.patel@in.ibm.com
License: Apache 2.0
Location: /Users/nelwin/Library/Python/3.10/lib/python/site-packages
Requires: ibm-cloud-sdk-core, pandas, python-dateutil, requests
Required-by: 


## Configure credentials

Provide your IBM Watson OpenScale credentials in the following cell:

In [5]:
WOS_CREDENTIALS = {
    "url": "",
    "username": "",
    "password": "",
    "instance_id": ""
}

## Specify model details

### Service provider and subscription metadata

In [6]:
# Service Provider

SERVICE_PROVIDER_NAME = ""
SERVICE_PROVIDER_DESCRIPTION = ""

# Subscription

SUBSCRIPTION_NAME = ""
SUBSCRIPTION_DESCRIPTION = ""

### IBM Analytics Engine - Spark

Make sure that the Apache Spark manager on IBM Analytics Engine is running, and then provide the following details:

- IAE_SPARK_DISPLAY_NAME: _Display Name of the Spark instance in IBM Analytics Engine_
- IAE_SPARK_JOBS_ENDPOINT: _Spark Jobs Endpoint for IBM Analytics Engine_
- IBM_CPD_VOLUME: _IBM Cloud Pak for Data storage volume name_
- IBM_CPD_USERNAME: _IBM Cloud Pak for Data username_
- IBM_CPD_APIKEY: _IBM Cloud Pak for Data API key_
- IAE_SPARK_DESCRIPTION: _Custom description for the Spark instance_

In [7]:
IAE_SPARK_DISPLAY_NAME = ""
IAE_SPARK_JOBS_ENDPOINT = ""
IBM_CPD_VOLUME = ""
IBM_CPD_USERNAME = ""
IBM_CPD_APIKEY = ""
IAE_SPARK_NAME = ""
IAE_SPARK_DESCRIPTION = ""

#### Provide Spark Resource Settings

To configure how much of your Spark Cluster resources this job can consume, edit the following values:

- max_num_executors: _Maximum Number of executors to launch for this session_
- min_executors: _Minimum Number of executors to launch for this session_
- executor_cores: _Number of cores to use for each executor_   
- executor_memory: _Amount of memory (in GBs) to use per executor process_
- driver_cores: _Number of cores to use for the driver process_
- driver_memory: _Amount of memory (in GBs) to use for the driver process_

In [8]:
spark_parameters = {
    "max_num_executors": 2,
    "min_num_executors": 1,
    "executor_cores": 3,
    "executor_memory": 2,
    "driver_cores": 2,
    "driver_memory": 2
}

### Storage Inputs

Please enter a name and description for your JDBC Storage

- JDBC_CONNECTION_NAME: _Custom display name for the JDBC Storage Connection_
- JDBC_CONNECTION_DESCRIPTION: _Custom description for the JDBC Storage Connection_

To connect to your JDBC storage, you must provide the following details:

 - JDBC_HOST: Hostname of the JDBC Connection
 - JDBC_PORT: Port of the JDBC Connection
 - JDBC_USE_SSL: Boolean Flag to indicate whether to use SSL while connecting.
 - JDBC_SSL_CERTIFICATE: SSL Certificate [Base64 encoded string] of the JDBC Connection. Ignored if JDBC_USE_SSL is False.
 - JDBC_DRIVER: Class name of the JDBC driver to use to connect.
 - JDBC_USERNAME: Username of the JDBC Connection
 - JDBC_PASSWORD: Password of the JDBC Connection
 - JDBC_DATABASE_NAME: Name of the Database to connect to.
 - JDBC_CONNECTION_TYPE: Type of the JDBC Connection.

In [9]:
JDBC_CONNECTION_NAME = ""
JDBC_CONNECTION_DESCRIPTION = ""

JDBC_CONNECTION_TYPE = "" # supported: db2, postgresql
JDBC_HOST = ""
JDBC_PORT = ""
JDBC_USE_SSL = ""
JDBC_SSL_CERTIFICATE = ""
JDBC_DRIVER = ""
JDBC_USERNAME = ""
JDBC_PASSWORD = ""
JDBC_DATABASE_NAME = ""

In [10]:
num_partitions_recommended = 12

if spark_parameters:
    executors = spark_parameters.get("max_num_executors", 2)
    cores = spark_parameters.get("executor_cores", 2)
    num_partitions_recommended = 2 * executors * cores
    
print("{} is the recommended value for number of partitions in your data. Please change this value as per your data.".format(num_partitions_recommended))

12 is the recommended value for number of partitions in your data. Please change this value as per your data.


- **PARTITION_COLUMN**: The column to help Spark read and write data using multiple workers in your JDBC storage. This will help improve the performance of your Spark jobs. The default value is set to `wos_partition_column`.
- **NUM_PARTITIONS**: The maximum number of partitions that Spark can divide the data into. In JDBC, it also means the maximum number of connections that Spark can make to the JDBC store for reading/writing data. 

The recommended value is calculated in the above cell as a multiple of total workers allotted for this job. You can use the same value or change it in the next cell.

In [11]:
PARTITION_COLUMN = "wos_partition_column"
NUM_PARTITIONS = num_partitions_recommended

### Feedback table metadata

The quality monitor stores metadata in the feedback table. To configure the quality monitor, you must provide the following details. To skip quality monitoring, run the following cell to initialize variables with the value of `None`.

- FEEDBACK_SCHEMA_NAME: _Schema name where feedback table is present_
- FEEDBACK_TABLE_NAME: _Name of the feedback table_

In [12]:
#feedback

FEEDBACK_SCHEMA_NAME = None
FEEDBACK_TABLE_NAME = None

### Payload and drift table metadata

The drift monitor stores metadata in the payload and drift tables. To configure the drift monitor, you must provide the following details. To skip drift monitoring, run the following cell to initialize variables with the value of `None`.

- PAYLOAD_SCHEMA_NAME: _Schema name where payload logging table is present_
- PAYLOAD_TABLE_NAME: _Name of the payload logging table_
- DRIFT_SCHEMA_NAME: _Schema name where drifted transactions table is present_
- DRIFT_TABLE_NAME: _Name of the drifted transactions table_

In [13]:
#payload logging

PAYLOAD_SCHEMA_NAME = None
PAYLOAD_TABLE_NAME = None

#drift

DRIFT_SCHEMA_NAME = None
DRIFT_TABLE_NAME = None

### Explainability table metadata

The explainability monitor requires the queue and result tables. The payload table can also be used as the queue table. To configure the explainability monitor, you must provide the following details. To skip explainability monitoring, run the following cell to initialize variables with the value of `None`.

- EXPLAINABILITY_SCHEMA_NAME: _Schema name where explanations queue, result tables are present_
- EXPLAINABILITY_QUEUE_TABLE_NAME: _Name of the explanations queue table_
- EXPLAINABILITY_RESULT_TABLE_NAME: _Name of the explanations result table_

In [14]:
#explainability

EXPLAINABILITY_DATABASE_NAME = None
EXPLAINABILITY_SCHEMA_NAME = None
EXPLAINABILITY_QUEUE_TABLE_NAME = None
EXPLAINABILITY_RESULT_TABLE_NAME = None

# 2. Configure Watson OpenScale <a name="openscale"></a>

### Import the required libraries and set up the Watson OpenScale client

In [15]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import *

authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS["url"],
        username=WOS_CREDENTIALS["username"],
        password=WOS_CREDENTIALS["password"],
        disable_ssl_verification=True
    )

wos_client = APIClient(authenticator=authenticator, service_url=WOS_CREDENTIALS["url"], service_instance_id=WOS_CREDENTIALS["instance_id"])

### Display Watson OpenScale datamart details

In [16]:
wos_client.data_marts.show()
data_marts = wos_client.data_marts.list().result.data_marts
data_mart_id = WOS_CREDENTIALS["instance_id"]

AIOSFASTPATHICP-00000000-0000-0000-0000-000000000000,Data Mart created by OpenScale ExpressPath,False,active,2024-11-28 19:42:06.911000+00:00,00000000-0000-0000-0000-000000000000


### Create a service provider

In [17]:
# Delete existing service provider with the same name as provided

service_providers = wos_client.service_providers.list().result.service_providers
for provider in service_providers:
    if provider.entity.name == SERVICE_PROVIDER_NAME:
        wos_client.service_providers.delete(service_provider_id=provider.metadata.id)
        break

In [18]:
# Add Service Provider
import time
time.sleep(5)
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.CUSTOM_MACHINE_LEARNING,
        credentials={},
        operational_space_id="production",
        background_mode=False
    ).result

service_provider_id = added_service_provider_result.metadata.id

wos_client.service_providers.show()




 Waiting for end of adding service provider 19f7e0e8-8e0a-4fc9-af3b-2a5e0359d66d 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




None,active,DriftV2Batch,custom_machine_learning,2025-01-17 12:52:49.763000+00:00,19f7e0e8-8e0a-4fc9-af3b-2a5e0359d66d
None,active,DRIFT-V2-BATCH-PROVIDER-CPD,custom_machine_learning,2025-01-13 06:14:05.063000+00:00,f8200240-5b59-4c2a-b80b-955ffea9708a
None,active,DriftV2 SDK Support,custom_machine_learning,2025-01-07 15:32:20.255000+00:00,652983c8-67e5-48d5-a852-fc686e08d49b
99999999-9999-9999-9999-999999999999,active,shreya-space,watson_machine_learning,2025-01-07 05:40:32.562000+00:00,aa11c920-1c84-4036-9ddf-e3ca53f62e25
None,active,DRIFT-V2-BATCH-PROVIDER,custom_machine_learning,2024-12-02 07:57:43.792000+00:00,4f82b928-48ec-49e1-a501-25920387e323
None,active,CUSTOM_APIKEY_CLOUD_WITHOUTAPI_PREPROD,custom_machine_learning,2024-12-02 05:35:44.191000+00:00,8f6039aa-cd0a-40b0-a9df-5fe9f4c4d129
None,active,CUSTOM_APIKEY_CLOUD_WITHOUTAPI,custom_machine_learning,2024-12-02 05:35:44.039000+00:00,f9d1dfaf-b3f6-44dd-866b-1e5ea358af4a
4ca7ec1c-6b35-48ef-a45c-0434c6985058,active,MRM_WMLV4_CLOUD_PREPROD,watson_machine_learning,2024-12-02 05:35:43.871000+00:00,a9b82a25-dd5b-473e-ac91-19ef0fcfe367
4ca7ec1c-6b35-48ef-a45c-0434c6985058,active,MRM_WMLV4_CLOUD_PROD,watson_machine_learning,2024-12-02 05:35:40.205000+00:00,4bae11c5-91a1-4bef-b711-721480161b6c
None,active,CUSTOM_HLS_PREPROD,custom_machine_learning,2024-12-02 05:35:29.287000+00:00,59cf73d9-9f7c-406f-9903-f0f21d6fe05d


Note: First 10 records were displayed.


In [19]:
service_provide_details = wos_client.service_providers.get(service_provider_id=service_provider_id).result
print(service_provide_details)

{
  "metadata": {
    "id": "19f7e0e8-8e0a-4fc9-af3b-2a5e0359d66d",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:service_provider:19f7e0e8-8e0a-4fc9-af3b-2a5e0359d66d",
    "url": "/v2/service_providers/19f7e0e8-8e0a-4fc9-af3b-2a5e0359d66d",
    "created_at": "2025-01-17T12:52:49.763000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "name": "DriftV2Batch",
    "service_type": "custom_machine_learning",
    "credentials": {
      "secret_id": "7f03d9f6-9ccd-4511-8bfa-b0b131041a20"
    },
    "operational_space_id": "production",
    "status": {
      "state": "active"
    }
  }
}


### Create integrated systems for Spark Engine and JDBC Storage

In [20]:
# Delete existing spark and jdbc integrated systems if present

integrated_systems = IntegratedSystems(wos_client).list().result.integrated_systems

for system in integrated_systems:
    if system.entity.name in (IAE_SPARK_NAME, JDBC_CONNECTION_NAME):
        print("Deleting integrated system {}".format(system.entity.name))
        IntegratedSystems(wos_client).delete(integrated_system_id=system.metadata.id)

Deleting integrated system SAMPLE_PG
Deleting integrated system IAEBatchSpark


#### Spark Engine

In [21]:
spark_engine_details = IntegratedSystems(wos_client).add(
    name=IAE_SPARK_NAME,
    description=IAE_SPARK_DESCRIPTION,
    type="spark",
    credentials={
        "username": IBM_CPD_USERNAME,
        "apikey": IBM_CPD_APIKEY
    },
    connection={
        "display_name": IAE_SPARK_DISPLAY_NAME,
        "endpoint": IAE_SPARK_JOBS_ENDPOINT,
        "volume": IBM_CPD_VOLUME,
        "location_type": "cpd_iae"
    }
).result

spark_engine_id = spark_engine_details.metadata.id
print(spark_engine_details)

{
  "metadata": {
    "id": "96cc81da-3473-4dfc-89c8-5a4db823c951",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:integrated_system:96cc81da-3473-4dfc-89c8-5a4db823c951",
    "url": "/v2/integrated_systems/96cc81da-3473-4dfc-89c8-5a4db823c951",
    "created_at": "2025-01-17T12:52:59.596000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "name": "IAEBatchSpark",
    "type": "spark",
    "description": "",
    "credentials": {
      "secret_id": "eb921b05-06bb-4669-b882-33d339e764ed"
    },
    "connection": {
      "display_name": "IAEBatchSpark",
      "endpoint": "https://cpd-cpd-instance.apps.osdev510pwx417gmc-1.cp.fyre.ibm.com/v4/analytics_engines/74dc7110-8603-4c64-961d-bdbb44296cfd/spark_applications",
      "location_type": "cpd_iae",
      "volume": "cpd-instance::IAEBatchTest"
    }
  }
}


#### JDBC Storage

In [22]:
jdbc_url = "jdbc:{}://{}:{}/{}".format(JDBC_CONNECTION_TYPE, JDBC_HOST, JDBC_PORT, JDBC_DATABASE_NAME)

jdbc_connection_details = IntegratedSystems(wos_client).add(
    name=JDBC_CONNECTION_NAME,
    description=JDBC_CONNECTION_DESCRIPTION,
    type="jdbc",
    credentials={
        "username": JDBC_USERNAME,
        "password": JDBC_PASSWORD,
        "jdbc_url": jdbc_url
    },
    connection={
        "location_type": "jdbc",
        "db_driver": JDBC_DRIVER,
        "use_ssl": JDBC_USE_SSL,
        "certificate": JDBC_SSL_CERTIFICATE,
    }
).result

jdbc_connection_id=jdbc_connection_details.metadata.id
print(jdbc_connection_details)

{
  "metadata": {
    "id": "abebe6fc-af0b-4a16-9d8f-cce0e5d15785",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:integrated_system:abebe6fc-af0b-4a16-9d8f-cce0e5d15785",
    "url": "/v2/integrated_systems/abebe6fc-af0b-4a16-9d8f-cce0e5d15785",
    "created_at": "2025-01-17T12:53:00.034000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "name": "SAMPLE_PG",
    "type": "jdbc",
    "description": "",
    "credentials": {
      "secret_id": "17b04597-0dfd-4f12-b8e2-1a3220b8ecc8"
    },
    "connection": {
      "certificate": "",
      "db_driver": "org.postgresql.Driver",
      "location_type": "jdbc",
      "use_ssl": false
    }
  }
}


# 3. Set up a subscription<a name="subscription"></a>

In [23]:
# Delete an existing subscription with the provided name

subscriptions = wos_client.subscriptions.list().result.subscriptions
for sub in subscriptions:
    if sub.entity.deployment.name == SUBSCRIPTION_NAME:
        wos_client.subscriptions.delete(subscription_id=sub.metadata.id)
        break

# Display all subscriptions
wos_client.subscriptions.show()

ac369058-f1cc-484d-bf13-f1a6d8062b53,model,[asset] Drift-Batch,00000000-0000-0000-0000-000000000000,02247a02-1d58-4b05-930b-f04ea167f33a,Drift-Batch,f8200240-5b59-4c2a-b80b-955ffea9708a,active,2025-01-16 09:35:45.225000+00:00,e436e53d-d385-4c15-84f3-1a0283920a62
5a84ca44-395b-4f1f-9238-28728148d146,model,[asset] Drift-Batch,00000000-0000-0000-0000-000000000000,1d7c4edd-2a05-4d49-9e59-c651e2a10aff,Drift-Batch,f8200240-5b59-4c2a-b80b-955ffea9708a,active,2025-01-16 05:46:47.478000+00:00,3e492d41-bba1-47d4-ab60-1a69090c36df
a0a602a0-9dae-42fd-92a3-f0be695c5f51,model,DriftV2Batch-SDK-Test,00000000-0000-0000-0000-000000000000,4c56bfa9-35a8-4169-9fed-db9f53b20cc2,DriftV2Batch-SDK-Test,652983c8-67e5-48d5-a852-fc686e08d49b,active,2025-01-07 15:34:07.611000+00:00,1cbc4b58-3374-4472-8556-841371719afa
49bbf658-2467-40df-b509-a26ab66818f7,model,DriftV2Batch-SDK-Test,00000000-0000-0000-0000-000000000000,95e02b12-e849-4a0f-8922-544dc1178837,DriftV2Batch-SDK-Test,652983c8-67e5-48d5-a852-fc686e08d49b,active,2025-01-07 15:33:24.779000+00:00,ad7b6384-d2a1-4dde-b66e-b081580cb5c4
f972ffb9-13c7-4e7b-8081-c400244231fa,model,GCR - P2 XGB Classifier - Model,00000000-0000-0000-0000-000000000000,d6777bec-f644-4068-a2bb-293675c312c8,gcr-shreya,aa11c920-1c84-4036-9ddf-e3ca53f62e25,active,2025-01-07 05:40:43.762000+00:00,4ea84707-a0e0-4ad3-a7a6-21c47fa82cec
fcd58b54-c281-449c-938f-8c20632d7671,model,GermanCreditRiskModelICP,00000000-0000-0000-0000-000000000000,d31e0810-b1ad-457f-bc91-a5aedf40d26c,GermanCreditRiskModelICP,85fcd6fc-754a-4ded-8cc2-969821bd4489,active,2024-12-30 06:36:07.118000+00:00,541e7e69-06e9-4a99-bde2-db84c5fb11bd
3ec26d2f-8859-43a3-a761-d8782b6ccd19,model,GermanCreditRiskModelPreProdICP,00000000-0000-0000-0000-000000000000,863dab2e-f054-457b-a488-0f30a66e93b9,GermanCreditRiskModelPreProdICP,bde1f060-acfa-4f1e-88a4-185c5887b290,active,2024-11-28 19:49:15.699000+00:00,afd32800-fcc6-4f46-a8c2-4aabd7faa079
133d36f3-4bb4-4b5e-8cc1-e20514abc087,model,GermanCreditRiskModelChallengerICP,00000000-0000-0000-0000-000000000000,10074385-1a1d-460b-875f-6717139810b9,GermanCreditRiskModelChallengerICP,bde1f060-acfa-4f1e-88a4-185c5887b290,active,2024-11-28 19:45:44.584000+00:00,4e642f07-755c-4031-b5cd-2e5259ecfd6b


### Set subscription metadata

In the following cell, type a path to the common configuration JSON file that you created by running the [common configuration notebook](https://github.com/IBM/watson-openscale-samples/blob/main/Cloud%20Pak%20for%20Data/Batch%20Support/Configuration%20generation%20for%20OpenScale%20batch%20subscription.ipynb). After you edit the path information, run the cell to set the asset details and properties, the deployment details, the analytics engine details, and to add the required tables as data sources.

In [24]:
import uuid

common_configuration = None

with open("/path/to/dir/containing/common_config.json", "r") as fp:
    configuration_json = json.load(fp)
    common_configuration = configuration_json.get("common_configuration")
    if common_configuration is None:
        print("Please provide the correct path to the common configuration JSON")
    
# Set asset details
asset = Asset(
    asset_id=str(uuid.uuid4()),
    url="",
    name=SUBSCRIPTION_NAME,
    asset_type=AssetTypes.MODEL,
    input_data_type=InputDataType.STRUCTURED,
    problem_type=ProblemType.BINARY_CLASSIFICATION
)

# Set deployment details
asset_deployment = AssetDeploymentRequest(
    deployment_id=str(uuid.uuid4()),
    name=SUBSCRIPTION_NAME,
    description=SUBSCRIPTION_DESCRIPTION,
    deployment_type="batch"
)

# Set asset properties 
asset_properties_request = AssetPropertiesRequest(
    label_column=common_configuration["label_column"],
    probability_fields=[common_configuration["probability"]],
    prediction_field=common_configuration["prediction"],
    feature_fields=common_configuration["feature_columns"],
    categorical_fields=common_configuration["categorical_columns"]
)

# Set analytics engine details
analytics_engine = AnalyticsEngine(
    type="spark",
    integrated_system_id=spark_engine_id,
    parameters = spark_parameters
)

# Add selected tables as data sources
data_sources = []
if FEEDBACK_SCHEMA_NAME is not None and FEEDBACK_TABLE_NAME is not None:
    feedback_data_source = DataSource(
        type="feedback",
        database_name=JDBC_DATABASE_NAME,
        schema_name=FEEDBACK_SCHEMA_NAME,
        table_name=FEEDBACK_TABLE_NAME,
        connection=DataSourceConnection(
            type="jdbc",
            integrated_system_id=jdbc_connection_id
        ),
        parameters={
            "partition_column": PARTITION_COLUMN,
            "num_partitions": NUM_PARTITIONS
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    data_sources.append(feedback_data_source)
    
if PAYLOAD_SCHEMA_NAME is not None and PAYLOAD_TABLE_NAME is not None \
    and DRIFT_SCHEMA_NAME is not None and DRIFT_TABLE_NAME is not None:
    payload_logging_data_source = DataSource(
        type="payload",
        database_name=JDBC_DATABASE_NAME,
        schema_name=PAYLOAD_SCHEMA_NAME,
        table_name=PAYLOAD_TABLE_NAME,
        connection=DataSourceConnection(
            type="jdbc",
            integrated_system_id=jdbc_connection_id
        ),
        parameters={
            "partition_column": PARTITION_COLUMN,
            "num_partitions": NUM_PARTITIONS
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    drifted_transactions_table_data_source = DataSource(
        type="drift",
        database_name=JDBC_DATABASE_NAME,
        schema_name=DRIFT_SCHEMA_NAME,
        table_name=DRIFT_TABLE_NAME,
        connection=DataSourceConnection(
            type="jdbc",
            integrated_system_id=jdbc_connection_id
        ),
        parameters={
            "partition_column": PARTITION_COLUMN,
            "num_partitions": NUM_PARTITIONS
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(payload_logging_data_source)
    data_sources.append(drifted_transactions_table_data_source)

if EXPLAINABILITY_DATABASE_NAME is not None and \
    EXPLAINABILITY_QUEUE_TABLE_NAME is not None and \
        EXPLAINABILITY_RESULT_TABLE_NAME is not None:

    explainability_queue_data_source = DataSource(
        type="explain_queue",
        database_name=EXPLAINABILITY_DATABASE_NAME,
        schema_name=EXPLAINABILITY_SCHEMA_NAME,
        table_name=EXPLAINABILITY_QUEUE_TABLE_NAME,
        connection=DataSourceConnection(
            type="jdbc", 
            integrated_system_id=jdbc_connection_id
        ),
        parameters={
            "partition_column": PARTITION_COLUMN,
            "num_partitions": NUM_PARTITIONS
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(explainability_queue_data_source)

    explainability_result_data_source = DataSource(
        type="explain_result",
        database_name=EXPLAINABILITY_DATABASE_NAME,
        schema_name=EXPLAINABILITY_SCHEMA_NAME,
        table_name=EXPLAINABILITY_RESULT_TABLE_NAME,
        connection=DataSourceConnection(
            type="jdbc",
            integrated_system_id=jdbc_connection_id
        ),
        parameters={},
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(explainability_result_data_source)

In [25]:
# Adding the subscription

subscription_details = Subscriptions(wos_client).add(
    data_mart_id=data_mart_id,
    service_provider_id=service_provider_id,
    asset=asset,
    deployment=asset_deployment,
    asset_properties=asset_properties_request,
    analytics_engine=analytics_engine,
    data_sources=data_sources).result

subscription_id = subscription_details.metadata.id
print(subscription_details)

{
  "metadata": {
    "id": "bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:subscription:bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04",
    "url": "/v2/subscriptions/bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04",
    "created_at": "2025-01-17T12:53:01.412000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "service_provider_id": "19f7e0e8-8e0a-4fc9-af3b-2a5e0359d66d",
    "asset": {
      "asset_id": "7870a295-3731-4c97-84e4-1a80e378825b",
      "url": "",
      "name": "GCR-DriftV2Batch",
      "asset_type": "model",
      "problem_type": "binary",
      "input_data_type": "structured"
    },
    "asset_properties": {
      "label_column": "Risk",
      "prediction_field": "prediction",
      "feature_fields": [
        "CheckingStatus",
        "LoanDuration",
        "CreditHistory",
        "LoanPurpose",
        "LoanAmount",
        "ExistingSa

In [26]:
# Add training, output, and input data schemas to the subscription
training_data_schema_patch_document=[
    JsonPatchOperation(op=OperationTypes.REPLACE, path="/asset_properties/training_data_schema", value=common_configuration["training_data_schema"])
]

input_data_schema_patch_document=[
    JsonPatchOperation(op=OperationTypes.REPLACE, path="/asset_properties/input_data_schema", value=common_configuration["input_data_schema"])
]

output_data_schema_patch_document=[
    JsonPatchOperation(op=OperationTypes.REPLACE, path="/asset_properties/output_data_schema", value=common_configuration["output_data_schema"])
]
wos_client.subscriptions.update(subscription_id=subscription_id, patch_document=training_data_schema_patch_document)
wos_client.subscriptions.update(subscription_id=subscription_id, patch_document=input_data_schema_patch_document)
wos_client.subscriptions.update(subscription_id=subscription_id, patch_document=output_data_schema_patch_document)


In [27]:
# Check subscription status
import time

subscription_status = None
while subscription_status not in ("active", "error"):
    subscription_status = wos_client.subscriptions.get(subscription_id).result.entity.status.state
    if subscription_status not in ("active", "error"):
        print(datetime.now().strftime("%H:%M:%S"), subscription_status)
        time.sleep(15)
        
print(datetime.now().strftime("%H:%M:%S"), subscription_status)

# print(wos_client.subscriptions.get(subscription_id).result.entity)

18:23:49 preparing
18:24:05 preparing
18:24:20 preparing
18:24:36 preparing
18:24:51 preparing
18:25:06 active


# 4. Quality monitoring <a name="quality"></a>

### Enable the quality monitor

In the following code cell, default values are set for the quality monitor. You can change the default values by updating the optional `min_feedback_data_size` attribute in the `parameters` dict and set the quality threshold in the `thresholds` list.

In [28]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    "min_feedback_data_size": 100
}

thresholds = [{
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
}]

quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

quality_monitor_instance_id = quality_monitor_details.metadata.id
print(quality_monitor_details)

{
  "metadata": {
    "id": "d9c5d805-a917-4e91-b32f-9d0c98ea27db",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:d9c5d805-a917-4e91-b32f-9d0c98ea27db",
    "url": "/v2/monitor_instances/d9c5d805-a917-4e91-b32f-9d0c98ea27db",
    "created_at": "2025-01-17T13:07:15.146000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "quality",
    "target": {
      "target_type": "subscription",
      "target_id": "bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04"
    },
    "parameters": {
      "min_feedback_data_size": 10
    },
    "thresholds": [
      {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "type": "relative",
        "delay_unit": "minute",
        "delay": 10
      },
      "rep

### Check monitor instance status

In [29]:
quality_status = None

while quality_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=quality_monitor_instance_id).result
    quality_status = monitor_instance_details.entity.status.state
    if quality_status not in ("active", "error"):
        print(datetime.now().strftime("%H:%M:%S"), quality_status)
        time.sleep(30)
        
print(datetime.now().strftime("%H:%M:%S"), quality_status)

18:37:15 preparing
18:37:46 active


In [30]:
monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=quality_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "d9c5d805-a917-4e91-b32f-9d0c98ea27db",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:d9c5d805-a917-4e91-b32f-9d0c98ea27db",
    "url": "/v2/monitor_instances/d9c5d805-a917-4e91-b32f-9d0c98ea27db",
    "created_at": "2025-01-17T13:07:15.146000Z",
    "created_by": "cpadmin",
    "modified_at": "2025-01-17T13:18:50.595000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "quality",
    "target": {
      "target_type": "subscription",
      "target_id": "bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04"
    },
    "parameters": {
      "min_feedback_data_size": 10
    },
    "thresholds": [
      {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "t

### Run an on-demand evaluation

In [31]:
# Check Quality monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=quality_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "d9c5d805-a917-4e91-b32f-9d0c98ea27db",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:d9c5d805-a917-4e91-b32f-9d0c98ea27db",
    "url": "/v2/monitor_instances/d9c5d805-a917-4e91-b32f-9d0c98ea27db",
    "created_at": "2025-01-17T13:07:15.146000Z",
    "created_by": "cpadmin",
    "modified_at": "2025-01-17T13:18:50.595000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "quality",
    "target": {
      "target_type": "subscription",
      "target_id": "bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04"
    },
    "parameters": {
      "min_feedback_data_size": 10
    },
    "thresholds": [
      {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "t

In [32]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "86f645aa-dd7d-41b3-8181-fdc131fea0f7",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:86f645aa-dd7d-41b3-8181-fdc131fea0f7",
    "url": "/v2/monitor_instances/d9c5d805-a917-4e91-b32f-9d0c98ea27db/runs/86f645aa-dd7d-41b3-8181-fdc131fea0f7",
    "created_at": "2025-01-17T13:27:23.535000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "min_feedback_data_size": 10
    },
    "status": {
      "state": "running",
      "queued_at": "2025-01-17T13:27:23.523000Z",
      "started_at": "2025-01-17T13:27:23.535000Z",
      "operators": []
    }
  }
}


In [33]:
# Check run status

quality_run_status = None
while quality_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=quality_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    quality_run_status = monitoring_run_details.entity.status.state
    if quality_run_status not in ("finished", "error"):
        print(datetime.now().strftime("%H:%M:%S"), quality_run_status)
        time.sleep(30)
print(datetime.now().strftime("%H:%M:%S"), quality_run_status)

18:57:27 running
18:57:57 running
18:58:27 running
18:58:58 running
18:59:28 running
19:00:12 running
19:00:43 running
19:01:14 running
19:01:44 running
19:02:15 running
19:02:45 finished


### Display quality metrics

In [34]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

2025-01-17 13:32:26.576418+00:00,true_positive_rate,d6316379-ff04-4988-86a5-5bc89b4c445f,0.9,None,None,['model_type:original'],quality,d9c5d805-a917-4e91-b32f-9d0c98ea27db,86f645aa-dd7d-41b3-8181-fdc131fea0f7,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-17 13:32:26.576418+00:00,area_under_roc,d6316379-ff04-4988-86a5-5bc89b4c445f,0.7227272727272727,0.8,None,['model_type:original'],quality,d9c5d805-a917-4e91-b32f-9d0c98ea27db,86f645aa-dd7d-41b3-8181-fdc131fea0f7,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-17 13:32:26.576418+00:00,precision,d6316379-ff04-4988-86a5-5bc89b4c445f,0.8780487804878049,None,None,['model_type:original'],quality,d9c5d805-a917-4e91-b32f-9d0c98ea27db,86f645aa-dd7d-41b3-8181-fdc131fea0f7,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-17 13:32:26.576418+00:00,f1_measure,d6316379-ff04-4988-86a5-5bc89b4c445f,0.888888888888889,None,None,['model_type:original'],quality,d9c5d805-a917-4e91-b32f-9d0c98ea27db,86f645aa-dd7d-41b3-8181-fdc131fea0f7,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-17 13:32:26.576418+00:00,accuracy,d6316379-ff04-4988-86a5-5bc89b4c445f,0.8235294117647058,None,None,['model_type:original'],quality,d9c5d805-a917-4e91-b32f-9d0c98ea27db,86f645aa-dd7d-41b3-8181-fdc131fea0f7,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-17 13:32:26.576418+00:00,log_loss,d6316379-ff04-4988-86a5-5bc89b4c445f,0.4702648897572855,None,None,['model_type:original'],quality,d9c5d805-a917-4e91-b32f-9d0c98ea27db,86f645aa-dd7d-41b3-8181-fdc131fea0f7,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-17 13:32:26.576418+00:00,false_positive_rate,d6316379-ff04-4988-86a5-5bc89b4c445f,0.45454545454545453,None,None,['model_type:original'],quality,d9c5d805-a917-4e91-b32f-9d0c98ea27db,86f645aa-dd7d-41b3-8181-fdc131fea0f7,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-17 13:32:26.576418+00:00,area_under_pr,d6316379-ff04-4988-86a5-5bc89b4c445f,0.8733620277379245,None,None,['model_type:original'],quality,d9c5d805-a917-4e91-b32f-9d0c98ea27db,86f645aa-dd7d-41b3-8181-fdc131fea0f7,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-17 13:32:26.576418+00:00,recall,d6316379-ff04-4988-86a5-5bc89b4c445f,0.9,None,None,['model_type:original'],quality,d9c5d805-a917-4e91-b32f-9d0c98ea27db,86f645aa-dd7d-41b3-8181-fdc131fea0f7,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04


# 5. Drift monitoring <a name="drift"></a>

### Enable the drift monitor

In the following code cell, type a path to the drift configuration tar ball.

In [35]:
wos_client.monitor_instances.upload_drift_model(
    model_path="drift_archive.tar.gz",
    data_mart_id=data_mart_id,
    subscription_id=subscription_id
).result

{'data_constraints': {'id': 'eb7b77e5-29ab-4ac9-a60e-92a0fbfc2acc',
  'version': '0.02_batch',
  'columns': [{'name': 'Age',
    'dtype': 'numeric_discrete',
    'count': 1000,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'CheckingStatus',
    'dtype': 'categorical',
    'count': 1000,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'CreditHistory',
    'dtype': 'categorical',
    'count': 1000,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'CurrentResidenceDuration',
    'dtype': 'numeric_discrete',
    'count': 1000,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'Dependents',
    'dtype': 'numeric_discrete',
    'count': 1000,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'EmploymentDuration',
    'dtype': 'categorical',
    'count': 1000,
    'appr

In the following code cell, default values are set for the drift monitor. You can change the default values by updating the values in the `parameters` section. The `min_samples` parameter controls the number of records that triggers the drift monitor to run. The `drift_threshold` parameter sets the threshold in decimal format for the drift percentage to trigger an alert. The `train_drift_model` parameter controls whether to re-train the model based on the drift analysis.

In [36]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)

Deleted existing drift monitor instance with id:  ae41323f-fcf9-4330-af70-4dce94853d1f


In [37]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    "min_samples": 100,
    "drift_threshold": 0.05,
    "train_drift_model": False
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
print(drift_monitor_details)

{
  "metadata": {
    "id": "8f44d3d8-30d4-40fe-96c5-e8d6abaf8466",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:8f44d3d8-30d4-40fe-96c5-e8d6abaf8466",
    "url": "/v2/monitor_instances/8f44d3d8-30d4-40fe-96c5-e8d6abaf8466",
    "created_at": "2025-01-17T13:40:25.023000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "drift",
    "target": {
      "target_type": "subscription",
      "target_id": "bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04"
    },
    "parameters": {
      "drift_threshold": 0.05,
      "min_samples": 10,
      "train_drift_model": false
    },
    "thresholds": [
      {
        "metric_id": "drift_magnitude",
        "type": "upper_limit",
        "value": 0.5
      },
      {
        "metric_id": "predicted_accuracy",
        "type": "upper_limit",
        "value": 0.8
      },
      {
        "metric_id": "data_dr

### Check monitor instance status

In [38]:
drift_status = None

while drift_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_monitor_instance_id).result
    drift_status = monitor_instance_details.entity.status.state
    if drift_status not in ("active", "error"):
        print(datetime.now().strftime("%H:%M:%S"), drift_status)
        time.sleep(30)

print(datetime.now().strftime("%H:%M:%S"), drift_status)

19:10:28 active


### Run an on-demand evaluation

In [39]:
# Check Drift monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "8f44d3d8-30d4-40fe-96c5-e8d6abaf8466",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:8f44d3d8-30d4-40fe-96c5-e8d6abaf8466",
    "url": "/v2/monitor_instances/8f44d3d8-30d4-40fe-96c5-e8d6abaf8466",
    "created_at": "2025-01-17T13:40:25.023000Z",
    "created_by": "cpadmin",
    "modified_at": "2025-01-17T13:40:26.052000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "drift",
    "target": {
      "target_type": "subscription",
      "target_id": "bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04"
    },
    "parameters": {
      "config_status": {
        "model_name": null,
        "state": "finished"
      },
      "data_drift_enabled": true,
      "data_drift_threshold": 0.1,
      "drift_buffer_range": [
        -4.5,
        4.5
      ],
      "drift_model_version": "spark-3.4.2",
      "drift_thre

In [40]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "96aa6acc-627a-4056-8347-d166f5447a3b",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:96aa6acc-627a-4056-8347-d166f5447a3b",
    "url": "/v2/monitor_instances/8f44d3d8-30d4-40fe-96c5-e8d6abaf8466/runs/96aa6acc-627a-4056-8347-d166f5447a3b",
    "created_at": "2025-01-17T13:40:30.194000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "config_status": {
        "model_name": null,
        "state": "finished"
      },
      "data_drift_enabled": true,
      "data_drift_threshold": 0.1,
      "drift_buffer_range": [
        -4.5,
        4.5
      ],
      "drift_model_version": "spark-3.4.2",
      "drift_threshold": 0.05,
      "min_samples": 10,
      "model_drift_enabled": true,
      "table_schema": {
        "fields": [
          {
            "length": 64,
            "metadata": {},
            "name": "scoring_id",
            "nullable": false,

In [41]:
# Check run status

drift_run_status = None
while drift_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=drift_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    drift_run_status = monitoring_run_details.entity.status.state
    if drift_run_status not in ("finished", "error"):
        print(datetime.now().strftime("%H:%M:%S"), drift_run_status)
        time.sleep(30)
        
print(datetime.now().strftime("%H:%M:%S"), drift_run_status)

19:10:31 running
19:11:01 running
19:11:31 running
19:12:02 running
19:12:32 running
19:13:03 running
19:13:33 running
19:14:03 running
19:14:34 running
19:15:04 running
19:15:35 running
19:16:05 running
19:16:35 running
19:17:06 running
19:17:36 finished


### Display drift metrics

In [42]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

2025-01-17 13:40:31.909545+00:00,data_drift_magnitude,69c11067-96a9-4d3c-bfdc-63a76eefae45,0.0,None,0.1,[],drift,8f44d3d8-30d4-40fe-96c5-e8d6abaf8466,96aa6acc-627a-4056-8347-d166f5447a3b,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-17 13:40:31.909545+00:00,drift_magnitude,69c11067-96a9-4d3c-bfdc-63a76eefae45,0.2860832137733142,None,0.05,[],drift,8f44d3d8-30d4-40fe-96c5-e8d6abaf8466,96aa6acc-627a-4056-8347-d166f5447a3b,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-17 13:40:31.909545+00:00,predicted_accuracy,69c11067-96a9-4d3c-bfdc-63a76eefae45,0.35294117647058826,None,None,[],drift,8f44d3d8-30d4-40fe-96c5-e8d6abaf8466,96aa6acc-627a-4056-8347-d166f5447a3b,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04


# 6. Drift V2 monitoring <a name="drift_v2"></a>

### Enable the drift v2 monitor

In the following code cell, type a path to the drift v2 configuration tar ball.

In [43]:
# Upload drift_v2 archive

wos_client.monitor_instances.upload_drift_v2_archive(
    archive_path="/path/to/dir/containing/drift_v2_archive.tar.gz",
    data_mart_id=data_mart_id,
    subscription_id=subscription_id
).result

In [44]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift_v2" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift v2 monitor instance with id: ', monitor_instance.metadata.id)

Deleted existing drift_v2 monitor instance with id:  96bfcb23-e442-4a64-9673-44e28d557076


In [45]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
        "min_samples": 100,
        "max_samples": 1000,
        "train_archive": False,
    }
drift_v2_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT_V2.ID,
    target=target,
    parameters=parameters
).result

drift_v2_monitor_instance_id = drift_v2_monitor_details.metadata.id
drift_v2_monitor_instance_id

'a59df637-dddc-4dd5-a9f9-9d5b7ee2cfa9'

### Check monitor instance status

In [46]:
drift_v2_status = None

while drift_v2_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_v2_monitor_instance_id).result
    drift_v2_status = monitor_instance_details.entity.status.state
    if drift_v2_status not in ("active", "error"):
        print(datetime.now().strftime("%H:%M:%S"), drift_v2_status)
        time.sleep(30)

print(datetime.now().strftime("%H:%M:%S"), drift_v2_status)

11:26:13 active


In [47]:
# Check Drift v2 monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_v2_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "a59df637-dddc-4dd5-a9f9-9d5b7ee2cfa9",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:a59df637-dddc-4dd5-a9f9-9d5b7ee2cfa9",
    "url": "/v2/monitor_instances/a59df637-dddc-4dd5-a9f9-9d5b7ee2cfa9",
    "created_at": "2025-01-17T16:16:24.057000Z",
    "created_by": "cpadmin",
    "modified_at": "2025-01-17T16:18:36.117000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "drift_v2",
    "target": {
      "target_type": "subscription",
      "target_id": "bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04"
    },
    "parameters": {
      "advanced_controls": {
        "enable_drift_v2_batch": true
      },
      "context": {
        "baseline": {
          "data_set_id": "e32f39c1-e782-4a55-89c6-66eb4e7bb3b1"
        },
        "copy_subscription": {
          "enabled": false,
          "source_id": null
   

In [48]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_v2_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "a0cd039f-99c6-4d3c-9229-cdecb4c218eb",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:a0cd039f-99c6-4d3c-9229-cdecb4c218eb",
    "url": "/v2/monitor_instances/a59df637-dddc-4dd5-a9f9-9d5b7ee2cfa9/runs/a0cd039f-99c6-4d3c-9229-cdecb4c218eb",
    "created_at": "2025-01-20T06:12:59.935000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "advanced_controls": {
        "enable_drift_v2_batch": true
      },
      "context": {
        "baseline": {
          "data_set_id": "e32f39c1-e782-4a55-89c6-66eb4e7bb3b1"
        },
        "copy_subscription": {
          "enabled": false,
          "source_id": null
        },
        "data_sets": {
          "drift_insights": "e859a19f-50ff-4a85-af2e-438221e2fbc0",
          "drift_intervals": "e40e985b-45cc-466a-bd1a-f24b8c5c6752",
          "drift_scores": "713247b6-11bc-49f7-9676-f0471e24f064",
          "drift_

In [49]:
# Check run status
drift_v2_run_status = None
while drift_v2_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=drift_v2_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    drift_v2_run_status = monitoring_run_details.entity.status.state
    if drift_v2_run_status not in ("finished", "error"):
        print(datetime.now().strftime("%H:%M:%S"), drift_v2_run_status)
        time.sleep(30)
        
print(datetime.now().strftime("%H:%M:%S"), drift_v2_run_status)



12:16:59 running
12:17:30 running
12:18:00 running
12:18:30 running
12:19:01 running
12:19:32 running
12:20:03 running
12:20:33 finished


### Check Drift V2 metrics

In [50]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_v2_monitor_instance_id)

2025-01-20 06:40:05.472457+00:00,records_processed,9a1cc567-aacc-4089-badc-f055dfd39831,38.0,None,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,a59df637-dddc-4dd5-a9f9-9d5b7ee2cfa9,a0cd039f-99c6-4d3c-9229-cdecb4c218eb,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-20 06:40:05.472457+00:00,confidence_drift_score,9a1cc567-aacc-4089-badc-f055dfd39831,0.9376,None,0.05,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,a59df637-dddc-4dd5-a9f9-9d5b7ee2cfa9,a0cd039f-99c6-4d3c-9229-cdecb4c218eb,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-20 06:40:05.472457+00:00,records_processed,9a1cc567-aacc-4089-badc-f055dfd39831,38.0,None,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,a59df637-dddc-4dd5-a9f9-9d5b7ee2cfa9,a0cd039f-99c6-4d3c-9229-cdecb4c218eb,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-20 06:40:05.472457+00:00,confidence_drift_score,9a1cc567-aacc-4089-badc-f055dfd39831,0.9217,None,0.05,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,a59df637-dddc-4dd5-a9f9-9d5b7ee2cfa9,a0cd039f-99c6-4d3c-9229-cdecb4c218eb,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-20 06:40:10.476230+00:00,records_processed,9f4767ed-085a-4dfe-9ff3-e8a81669068f,38.0,None,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,a59df637-dddc-4dd5-a9f9-9d5b7ee2cfa9,a0cd039f-99c6-4d3c-9229-cdecb4c218eb,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-20 06:40:10.476230+00:00,confidence_drift_score,9f4767ed-085a-4dfe-9ff3-e8a81669068f,0.9784,None,0.05,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,a59df637-dddc-4dd5-a9f9-9d5b7ee2cfa9,a0cd039f-99c6-4d3c-9229-cdecb4c218eb,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-20 06:40:10.476230+00:00,records_processed,9f4767ed-085a-4dfe-9ff3-e8a81669068f,38.0,None,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,a59df637-dddc-4dd5-a9f9-9d5b7ee2cfa9,a0cd039f-99c6-4d3c-9229-cdecb4c218eb,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-20 06:40:10.476230+00:00,confidence_drift_score,9f4767ed-085a-4dfe-9ff3-e8a81669068f,0.9729,None,0.05,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,a59df637-dddc-4dd5-a9f9-9d5b7ee2cfa9,a0cd039f-99c6-4d3c-9229-cdecb4c218eb,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-20 06:40:13.737012+00:00,records_processed,42ce2979-323f-45ca-97f1-950a42a4084d,38.0,None,None,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:class', 'field_name:prediction']",drift_v2,a59df637-dddc-4dd5-a9f9-9d5b7ee2cfa9,a0cd039f-99c6-4d3c-9229-cdecb4c218eb,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-20 06:40:13.737012+00:00,prediction_drift_score,42ce2979-323f-45ca-97f1-950a42a4084d,0.0421,None,0.05,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:class', 'field_name:prediction']",drift_v2,a59df637-dddc-4dd5-a9f9-9d5b7ee2cfa9,a0cd039f-99c6-4d3c-9229-cdecb4c218eb,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04


Note: First 10 records were displayed.


# 7. Fairness monitoring <a name="fairness"></a>

### Enable the fairness monitor

The following code cell, will enable the fairness monitor.

In [51]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = configuration_json["fairness_configuration"]["parameters"]
thresholds = configuration_json["fairness_configuration"]["thresholds"]

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

fairness_monitor_instance_id = fairness_monitor_details.metadata.id
print(fairness_monitor_details)

{
  "metadata": {
    "id": "21d33693-6376-4769-9d94-99f1415a48c6",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:21d33693-6376-4769-9d94-99f1415a48c6",
    "url": "/v2/monitor_instances/21d33693-6376-4769-9d94-99f1415a48c6",
    "created_at": "2025-01-20T07:00:22.442000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "fairness",
    "target": {
      "target_type": "subscription",
      "target_id": "bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04"
    },
    "parameters": {
      "training_data_last_processed_time": "2025-01-17T09:40:55.219953Z",
      "min_records": 100,
      "features": [
        {
          "minority": [
            "female"
          ],
          "feature": "Sex",
          "metric_ids": [
            "fairness_value",
            "statistical_parity_difference",
            "average_odds_difference",
            "av

### Check monitor instance status

In [52]:
fairness_state = fairness_monitor_details.entity.status.state

while fairness_state not in ("active", "error"):
    print(datetime.utcnow().strftime('%H:%M:%S'), fairness_state)
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=fairness_monitor_instance_id).result
    fairness_state = monitor_instance_details.entity.status.state
    time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), fairness_state)

07:00:26 preparing
07:00:58 active


### Run an on-demand evaluation

In [53]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id
print(monitoring_run_details)

{
  "metadata": {
    "id": "dc01533c-53fa-4a61-88d5-e5c33af39b39",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:dc01533c-53fa-4a61-88d5-e5c33af39b39",
    "url": "/v2/monitor_instances/21d33693-6376-4769-9d94-99f1415a48c6/runs/dc01533c-53fa-4a61-88d5-e5c33af39b39",
    "created_at": "2025-01-20T07:07:47.594000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "favourable_class": [
        "No Risk"
      ],
      "features": [
        {
          "feature": "Sex",
          "majority": [
            "male"
          ],
          "metric_ids": [
            "fairness_value",
            "statistical_parity_difference",
            "average_odds_difference",
            "average_abs_odds_difference",
            "false_negative_rate_difference",
            "false_positive_rate_difference",
            "false_discovery_rate_difference",
            "false_omission_rate_differenc

In [54]:
# Check run status

fairness_run_status = monitoring_run_details.entity.status.state
while fairness_run_status not in ("finished", "error"):
    print(datetime.utcnow().strftime("%H:%M:%S"), fairness_run_status)
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=fairness_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    fairness_run_status = monitoring_run_details.entity.status.state
    time.sleep(30)
        
print(datetime.utcnow().strftime("%H:%M:%S"), fairness_run_status)

07:07:51 running
07:08:22 running
07:08:53 running
07:09:24 running
07:09:54 running
07:10:25 running
07:10:56 running
07:11:26 running
07:11:57 running
07:12:27 running
07:12:58 running
07:13:29 finished


### Display fairness metrics

In [55]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

2025-01-20 07:12:50.792485+00:00,false_positive_rate_difference,167d2bea-0fcb-4b82-a641-b8d2e0754a70,0.857,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,21d33693-6376-4769-9d94-99f1415a48c6,dc01533c-53fa-4a61-88d5-e5c33af39b39,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-20 07:12:50.792485+00:00,false_negative_rate_difference,167d2bea-0fcb-4b82-a641-b8d2e0754a70,-0.174,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,21d33693-6376-4769-9d94-99f1415a48c6,dc01533c-53fa-4a61-88d5-e5c33af39b39,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-20 07:12:50.792485+00:00,error_rate_difference,167d2bea-0fcb-4b82-a641-b8d2e0754a70,0.023,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,21d33693-6376-4769-9d94-99f1415a48c6,dc01533c-53fa-4a61-88d5-e5c33af39b39,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-20 07:12:50.792485+00:00,false_omission_rate_difference,167d2bea-0fcb-4b82-a641-b8d2e0754a70,-0.4,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,21d33693-6376-4769-9d94-99f1415a48c6,dc01533c-53fa-4a61-88d5-e5c33af39b39,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-20 07:12:50.792485+00:00,fairness_value,167d2bea-0fcb-4b82-a641-b8d2e0754a70,150.00000000000003,95.0,125.0,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,21d33693-6376-4769-9d94-99f1415a48c6,dc01533c-53fa-4a61-88d5-e5c33af39b39,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-20 07:12:50.792485+00:00,statistical_parity_difference,167d2bea-0fcb-4b82-a641-b8d2e0754a70,0.333,-0.3,0.3,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,21d33693-6376-4769-9d94-99f1415a48c6,dc01533c-53fa-4a61-88d5-e5c33af39b39,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-20 07:12:50.792485+00:00,false_discovery_rate_difference,167d2bea-0fcb-4b82-a641-b8d2e0754a70,0.14,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,21d33693-6376-4769-9d94-99f1415a48c6,dc01533c-53fa-4a61-88d5-e5c33af39b39,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-20 07:12:50.792485+00:00,average_odds_difference,167d2bea-0fcb-4b82-a641-b8d2e0754a70,0.516,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,21d33693-6376-4769-9d94-99f1415a48c6,dc01533c-53fa-4a61-88d5-e5c33af39b39,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-20 07:12:50.792485+00:00,average_abs_odds_difference,167d2bea-0fcb-4b82-a641-b8d2e0754a70,0.516,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,21d33693-6376-4769-9d94-99f1415a48c6,dc01533c-53fa-4a61-88d5-e5c33af39b39,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04
2025-01-20 07:12:50.792485+00:00,false_positive_rate_difference,167d2bea-0fcb-4b82-a641-b8d2e0754a70,-0.455,-0.15,0.15,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:18-25']",fairness,21d33693-6376-4769-9d94-99f1415a48c6,dc01533c-53fa-4a61-88d5-e5c33af39b39,subscription,bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04


Note: First 10 records were displayed.


# 8. Explainability monitoring <a name="explainability"></a>

#### Upload explainability configuration archive
In the following code cell, type the path to the explainability configuration archive tar ball.

In [56]:
with open("/path/to/dir/containing/explainability.tar.gz", mode="rb") as explainability_tar:
    wos_client.monitor_instances.upload_explainability_archive(subscription_id=subscription_id, archive=explainability_tar)

print("Uploaded explainability configuration archive successfully.")

Uploaded explainability configuration archive successfully.


In [57]:
# Delete existing explainability monitor.
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "explainability" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing explainability monitor instance with id: ', monitor_instance.metadata.id)

Deleted existing explainability monitor instance with id:  5fc73737-02b7-4567-a0fe-4c8b2d8079ae


### Enable the explainability monitor

In [58]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    # Comment the below lines to disable lime global explanation. 
    # Lime global explanation is available from Cloud Pak for Data version 4.6.4 onwards.
    "global_explanation": {
        "enabled": False,  # Flag to enable global explanation 
        "explanation_method": "lime",
        # "sample_size": 1000, # [Optional] The sample size of records to be used for generating payload data global explanation. If not specified entire data in the payload window is used.
    }
}

explainability_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_instance_id = explainability_monitor_details.metadata.id
print(explainability_monitor_details)

{
  "metadata": {
    "id": "816096f0-61c0-44c2-8823-3df0968336c0",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:816096f0-61c0-44c2-8823-3df0968336c0",
    "url": "/v2/monitor_instances/816096f0-61c0-44c2-8823-3df0968336c0",
    "created_at": "2025-01-20T07:16:55.384000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "explainability",
    "target": {
      "target_type": "subscription",
      "target_id": "bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04"
    },
    "parameters": {
      "global_explanation": {
        "enabled": false,
        "explanation_method": "lime"
      }
    },
    "thresholds": [
      {
        "metric_id": "global_explanation_stability",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "typ

#### Upload explainability configuration archive
In the following code cell, type the path to the explainability configuration archive tar ball.

### Check monitor instance status

In [59]:
explainability_status = None

while explainability_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=explainability_monitor_instance_id).result
    explainability_status = monitor_instance_details.entity.status.state
    if explainability_status not in ("active", "error"):
        print(datetime.utcnow().strftime('%H:%M:%S'), explainability_status)
        time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), explainability_status)

07:34:50 preparing
07:35:20 preparing
07:35:51 preparing
07:36:21 preparing
07:36:52 preparing
07:37:22 preparing
07:37:53 active


### Run an on-demand evaluation

In [60]:
# Check Explainbility monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=explainability_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "816096f0-61c0-44c2-8823-3df0968336c0",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:816096f0-61c0-44c2-8823-3df0968336c0",
    "url": "/v2/monitor_instances/816096f0-61c0-44c2-8823-3df0968336c0",
    "created_at": "2025-01-20T07:16:55.384000Z",
    "created_by": "cpadmin",
    "modified_at": "2025-01-20T07:51:32.954000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "explainability",
    "target": {
      "target_type": "subscription",
      "target_id": "bf74eab0-6e49-4c9b-a68a-d1bf3f7dcb04"
    },
    "parameters": {
      "config_modified_at": "2025-01-20T07:51:32.821500Z",
      "config_package_file": "explainability.tar.gz",
      "controllable_features": [],
      "explanations_count": {
        "failed": 0,
        "total": 0
      },
      "global_explanation": {
        "enabled": 

In [61]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id="816096f0-61c0-44c2-8823-3df0968336c0").result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "c60d12cd-e9d8-4fe0-851e-dc8bcfecc08f",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:c60d12cd-e9d8-4fe0-851e-dc8bcfecc08f",
    "url": "/v2/monitor_instances/816096f0-61c0-44c2-8823-3df0968336c0/runs/c60d12cd-e9d8-4fe0-851e-dc8bcfecc08f",
    "created_at": "2025-01-20T08:52:34.778000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "triggered_by": "user",
    "parameters": {
      "config_modified_at": "2025-01-20T07:51:32.821500Z",
      "config_package_file": "explainability.tar.gz",
      "controllable_features": [],
      "explanations_count": {
        "failed": 0,
        "total": 0
      },
      "global_explanation": {
        "enabled": false,
        "explanation_method": "lime"
      },
      "lime": {
        "perturbations_count": 10000
      },
      "validate_table_job_app_id": null,
      "validate_table_job_id": "02410e33-81a0-4691-b2c0-db745075cca0",
      "validate_table_job_outp

In [62]:
# Check run status

explainability_run_status = None
while explainability_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=explainability_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    explainability_run_status = monitoring_run_details.entity.status.state
    if explainability_run_status not in ("finished", "error"):
        print(datetime.utcnow().strftime("%H:%M:%S"), explainability_run_status)
        time.sleep(60)
        
print(datetime.utcnow().strftime("%H:%M:%S"), explainability_run_status)

08:52:45 finished


In [63]:
# View the global explanation stability metric. When lime global explanation is enabled, the monitor run computes global explanation and publishes global_explanation_stability metric.
wos_client.monitor_instances.show_metrics(monitor_instance_id=explainability_monitor_instance_id)

2025-01-21 09:28:51.894207+00:00,global_explanation_stability,36c987bd-9b8c-41c4-9df7-3bb2350c22a6,0.9795605604404594,0.8,None,['data_set_type:payload'],explainability,bfa372bb-c0dc-4aee-b586-6796a158e4b3,bc4264e4-204e-49e1-a2b8-f6641e2c4da8,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86
2025-01-21 09:21:45.761430+00:00,global_explanation_stability,f4714c1f-af0f-4469-bdd6-053bf0a875e8,1.0,0.8,None,['data_set_type:training'],explainability,bfa372bb-c0dc-4aee-b586-6796a158e4b3,9ac9471e-41e8-4348-aa78-067a5753ea3c,subscription,52d2745c-45f4-4edc-8e5c-9ea6dacd2a86


### Display sample explanations

In [64]:
explanations = wos_client.monitor_instances.get_all_explaination_tasks(subscription_id=subscription_id).result
print(explanations)

{
  "total_count": 100,
  "limit": 50,
  "offset": 0,
  "explanation_fields": [
    "explanation_task_id",
    "scoring_id",
    "created_at",
    "finished_at",
    "status",
    "prediction",
    "subscription_id",
    "deployment_id",
    "asset_name",
    "deployment_name",
    "probability",
    "explanation_type"
  ],
  "explanation_values": [
    [
      "3378de3d-ee19-410f-9c97-263fb1a2946f",
      "9820ff2a017cadb6ada7931f23adf6f7-1",
      "2025-01-21T09:24:34.974443Z",
      "2025-01-21T09:24:34.974614Z",
      "finished",
      "No Risk",
      "52d2745c-45f4-4edc-8e5c-9ea6dacd2a86",
      "fbb6b55b-d40c-4b8b-802b-8a6448450b73",
      "GCR-DriftV2-2",
      "GCR-DriftV2-2",
      0.677657,
      "lime"
    ],
    [
      "fda2b010-4ef4-43e4-b73f-ebe386e58cff",
      "96927a161a0d60c579981e93f8d78fc2-7",
      "2025-01-21T09:24:34.993858Z",
      "2025-01-21T09:24:34.994006Z",
      "finished",
      "No Risk",
      "52d2745c-45f4-4edc-8e5c-9ea6dacd2a86",
      "fbb6b55b-d4

## Congratulations!

You have finished the Batch demo for IBM Watson OpenScale by using Apache Spark on Cloud Pak for Data IBM Analytics Engine. You can now view the [Watson OpenScale Dashboard](https://url-to-your-cp4d-cluster/aiopenscale). Click the tile for the **German Credit model** to see the quality, drift and fairness monitors. Click the timeseries graph to get detailed information on transactions during a specific time window.